In [78]:
from __future__ import  print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.set_random_seed(2019)

In [90]:
tf.reset_default_graph()
sess.close()

In [91]:
from sklearn.datasets import  load_iris
X,Y = load_iris(return_X_y=True)
x_data = tf.constant(X[:,0:4],name='x',dtype=tf.float32)
y_data = tf.expand_dims(tf.constant(Y, dtype=tf.float32),axis=-1)
w = tf.get_variable(initializer=tf.random_normal_initializer(seed=2019), shape=(4,1),dtype=tf.float32,name="weight")
b = tf.get_variable(initializer=tf.zeros_initializer(), shape=1,dtype=tf.float32, name="bias")

def logistic_regression(x):
    return tf.sigmoid(tf.matmul(x,w)+b)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
def binary_loss(y_pred,y):
    logit = tf.reduce_mean(y*tf.log(y_pred)+(1-y)*tf.log(1-y_pred))
    return -logit

y_pred = logistic_regression(x_data)

In [101]:
loss = binary_loss(y_pred, y_data)
#手动计算梯度，并更新
w_grad, b_grad = tf.gradients(loss,[w,b])
lr = 0.1
w_update = w.assign_sub(lr*w_grad)
b_update = b.assign_sub(lr*b_grad)

In [105]:
sess.run([w_update, b_update])
print(loss.eval(session=sess))

[array([[ 0.21858457],
        [-0.7079771 ],
        [ 0.86685777],
        [ 0.9065998 ]], dtype=float32), array([0.03749988], dtype=float32)]

In [115]:
#集成优化器
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1, name='optimizer')
train_op = optimizer.minimize(loss)
import time
sess.run(tf.global_variables_initializer())

start = time.time()
for e in range(1000):
    sess.run(train_op)
    if (e+1) %200 == 0:
        y_true_label = y_data.eval(session=sess)
        y_pred_numpy = y_pred.eval(session=sess)
        y_pred_label = np.greater_equal(y_pred_numpy,0.5).astype(np.float32)
        
        accuracy = np.mean(y_pred_label == y_true_label)
        loss_numpy = loss.eval(session=sess)
        print('Epoch %d, Loss: %.4f, Acc: %.4f,' % (e+1, loss_numpy, accuracy))

print('mamual_GD cost time: %.4f' % (time.time() - start))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in greater_equal
  del sys.path[0]


Epoch 200, Loss: nan, Acc: 0.3333,
Epoch 400, Loss: nan, Acc: 0.3333,
Epoch 600, Loss: nan, Acc: 0.3333,
Epoch 800, Loss: nan, Acc: 0.3333,
Epoch 1000, Loss: nan, Acc: 0.3333,
mamual_GD cost time: 0.4175


In [116]:
#tf 集成losses
loss1 = tf.losses.log_loss(predictions=y_pred, labels=y_data)
optimizer1 = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op1 = optimizer1.minimize(loss1)
sess.run(tf.global_variables_initializer())

start = time.time()
for e in range(1000):
    sess.run(train_op1)
    if (e+1) %200 == 0:
        y_true_label = y_data.eval(session=sess)
        y_pred_numpy = y_pred.eval(session=sess)
        y_pred_label = np.greater_equal(y_pred_numpy,0.5).astype(np.float32)
        
        accuracy = np.mean(y_pred_label == y_true_label)
        loss_numpy = loss.eval(session=sess)
        print('Epoch %d, Loss: %.4f, Acc: %.4f,' % (e+1, loss_numpy, accuracy))

print('mamual_GD cost time: %.4f' % (time.time() - start))